In [1]:
# Step0 (GPU-Diagnose & robustes Setup): Seeds, GPU-Check, Pfade, 'src'-Import
import os, sys, random, logging
from pathlib import Path
import numpy as np
import tensorflow as tf

# deterministische Seeds
os.environ["PYTHONHASHSEED"] = "0"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# Logging & TensorFlow-Verbosity reduzieren
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger("lstm_notebook")
tf.get_logger().setLevel("ERROR")

# GPU-Diagnose (zeigt, ob TF mit CUDA gebaut wurde und welche Geräte sichtbar sind)
print("TensorFlow-Version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))
print("Phys. GPUs:", tf.config.list_physical_devices("GPU"))
print("Log. GPUs :", tf.config.list_logical_devices("GPU"))

# Speicherwachstum setzen (falls GPU vorhanden)
for g in tf.config.list_physical_devices("GPU"):
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

# Projekt-Root finden (verhindert notebooks\artifacts-Falle)
def find_project_root(start: Path) -> Path:
    start = start.resolve()
    chain = [start, *start.parents]
    for p in chain:
        if (p / "artifacts" / "data" / "features_monthly.parquet").exists():
            return p
    for p in chain:
        if (p / "artifacts" / "data").exists():
            return p
    for p in chain:
        if (p / "config").exists() and (p / "src").exists():
            return p
    raise AssertionError("Project root not found – 'artifacts/data' oder 'config'+'src' erwartet.")

ROOT = find_project_root(Path.cwd())
ARTIFACTS     = ROOT / "artifacts"
DATA_DIR      = ARTIFACTS / "data"
CONF_DIR      = ROOT / "config"
FORECASTS_DIR = ARTIFACTS / "forecasts"
METRICS_DIR   = ARTIFACTS / "metrics"
REPORTS_DIR   = ARTIFACTS / "reports"
for p in [DATA_DIR, CONF_DIR, FORECASTS_DIR, METRICS_DIR, REPORTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Pfade für 'src' registrieren
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))
if str(ROOT / "src") not in sys.path: sys.path.insert(0, str(ROOT / "src"))
try:
    import importlib; importlib.import_module("src")
    logger.info("src-Paket importierbar.")
except Exception as e:
    logger.warning(f"'src' nicht importierbar (Diagnose): {e}")

logger.info(f"ROOT={ROOT} | DATA_DIR={DATA_DIR}")


INFO: src-Paket importierbar.
INFO: ROOT=C:\Users\gamer\Desktop\AktienPrognose | DATA_DIR=C:\Users\gamer\Desktop\AktienPrognose\artifacts\data


TensorFlow-Version: 2.20.0
Built with CUDA: False
CUDA_VISIBLE_DEVICES: None
Phys. GPUs: []
Log. GPUs : []


In [2]:
# Step1 (final): Features laden (Parquet/CSV-Fallback), Datumsindex setzen, Feature-Sets definieren
import pandas as pd
from pathlib import Path

# --- Feature-Spalten definieren ---
tech_cols = ["Return_Lag1", "3M_SMA_Return", "12M_SMA_Return", "3M_Momentum", "Volatility_6M"]
macro_cols = ["FedFunds_Delta_bps", "Inflation_YoY_pct", "UnemploymentRate", "VIX", "EPU_US", "FSI", "Gold_USD_oz", "WTI_Spot", "USD_per_EUR"]
integrated_cols = tech_cols + macro_cols

def load_features_df(data_dir: Path) -> pd.DataFrame:
    pq  = data_dir / "features_monthly.parquet"
    csv = data_dir / "features_monthly.csv"
    if pq.exists():
        df_ = pd.read_parquet(pq)
        logger.info(f"Geladen: {pq}")
        return df_
    if csv.exists():
        df_ = pd.read_csv(csv)
        logger.info(f"Geladen: {csv}")
        return df_
    avail = sorted([p.name for p in data_dir.glob("*")])
    raise FileNotFoundError(f"Kein features_monthly.[parquet|csv] in {data_dir}. Gefunden: {avail}")

df = load_features_df(DATA_DIR)

# --- Datumsindex sicherstellen ---
if not isinstance(df.index, pd.DatetimeIndex):
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df = df.set_index("date").sort_index()
    else:
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

print("Spalten (erste 20):", list(df.columns)[:20])
print("Zeitraum:", df.index.min().date(), "→", df.index.max().date(), "| n=", len(df))


INFO: Geladen: C:\Users\gamer\Desktop\AktienPrognose\artifacts\data\features_monthly.parquet


Spalten (erste 20): ['Return_Lag1', '3M_SMA_Return', '12M_SMA_Return', '3M_Momentum', 'Volatility_6M', 'FedFunds_Delta_bps', 'Inflation_YoY_pct', 'UnemploymentRate', 'VIX', 'EPU_US', 'FSI', 'Gold_USD_oz', 'WTI_Spot', 'USD_per_EUR', 'y_return_next_pct', 'y_direction_next']
Zeitraum: 2009-02-28 → 2025-05-31 | n= 196


In [3]:
# Step2: Zeitliche Aufteilung in Trainings- und Test-Daten
from src.utils.splits import time_split, split_Xy

# Konventioneller Split: Training bis 2019-12, Test ab 2020-01
train_df, test_df = time_split(df, train_end="2019-12-31", test_start="2020-01-31")
Xtr_full, ytr_reg, ytr_clf = split_Xy(train_df, y_reg="y_return_next_pct", y_clf="y_direction_next")
Xte_full, yte_reg, yte_clf = split_Xy(test_df,  y_reg="y_return_next_pct", y_clf="y_direction_next")

print("Train:", train_df.index.min().date(), "→", train_df.index.max().date(), "| n=", len(train_df))
print("Test :", test_df.index.min().date(), "→", test_df.index.max().date(), "| n=", len(test_df))


Train: 2009-02-28 → 2019-12-31 | n= 131
Test : 2020-01-31 → 2025-05-31 | n= 65


In [4]:
# Step3 (SAFE): TSCV, OOF, Platt-Kalibrierung (pure NumPy), Threshold, bestes k, finales Train
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score

# --- Stabilitäts-Tweaks (kein SciPy) ---
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
try:
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)
except Exception:
    pass
tf.keras.backend.clear_session()

# --- Sequenzbau ---
def build_sequences_matrix(X: np.ndarray, y: np.ndarray, k: int):
    Xs, ys = [], []
    for i in range(k-1, len(X)):
        Xs.append(X[i-k+1:i+1])
        ys.append(y[i])
    return np.asarray(Xs, dtype=np.float32), np.asarray(ys, dtype=np.int32)

# --- LSTM-Factory ---
def build_lstm_model(t_steps: int, n_feats: int) -> tf.keras.Model:
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(t_steps, n_feats)),
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer="adam")
    return model

# --- Platt-Kalibrierung (pure NumPy): p -> sigmoid(a*p + b) ---
def platt_fit(p: np.ndarray, y: np.ndarray, lr=0.05, l2=1e-4, max_iter=2000):
    # p und y als float32 / int32
    p = p.astype(np.float32).clip(1e-6, 1-1e-6)
    y = y.astype(np.float32)
    a = 0.0
    b = 0.0
    for it in range(max_iter):
        z = a * p + b
        q = 1.0 / (1.0 + np.exp(-z))
        # Gradienten (Log-Loss) + L2 nur auf 'a'
        grad_a = np.mean((q - y) * p) + l2 * a
        grad_b = np.mean(q - y)
        a -= lr * grad_a
        b -= lr * grad_b
        # primitive Konvergenzprüfung (optional)
        if it % 400 == 0:
            loss = -np.mean(y*np.log(q+1e-12) + (1-y)*np.log(1-q+1e-12)) + 0.5*l2*(a*a)
    return float(a), float(b)

def platt_predict(p: np.ndarray, ab: tuple[float,float]) -> np.ndarray:
    a, b = ab
    p = p.astype(np.float32).clip(1e-6, 1-1e-6)
    z = a * p + b
    q = 1.0 / (1.0 + np.exp(-z))
    return q.astype(np.float32)

# --- Wrapper: kalibrierte Probas (kann später auch sklearn-Calibrator akzeptieren) ---
def calibrate_proba(p_raw: np.ndarray, calibrator):
    if isinstance(calibrator, tuple) and len(calibrator) == 2:
        return platt_predict(p_raw, calibrator)
    # Fallback, falls mal ein sklearn-Calibrator übergeben wird
    return calibrator.predict_proba(p_raw.reshape(-1,1))[:,1]

# --- Hyperparameter & Container ---
ks = [6, 9, 12]
feature_sets = {
    "TECH": tech_cols,
    "MACRO": macro_cols,
    "INTEGRATED": integrated_cols
}
results = {}

for fs_name, cols in feature_sets.items():
    print(f"\n>>> Feature-Set: {fs_name}")
    Xtr = Xtr_full[cols].values.astype(np.float32)
    ytr = ytr_clf.values.astype(np.int32)

    best = {"k": None, "auc": -np.inf, "calibrator": None, "threshold": None,
            "oof_f1": None, "oof_auc": None, "history": None}

    for k in ks:
        y_oof, p_oof = [], []

        # Expanding 5-Fold: Val=12, Embargo=1
        initial_train_end = max(k-1, 65)
        embargo = 1
        val_size = 12
        last_history = None

        for fold in range(5):
            train_end_idx = initial_train_end + fold*(val_size+embargo)
            val_start_idx = train_end_idx + 1 + embargo
            val_end_idx   = min(val_start_idx + val_size - 1, len(Xtr)-1)
            if val_start_idx >= len(Xtr) or (val_end_idx - (k-1)) < 0:
                break

            X_tr = Xtr[:train_end_idx+1]
            y_tr = ytr[:train_end_idx+1]
            X_val = Xtr[val_start_idx:val_end_idx+1]
            y_val = ytr[val_start_idx:val_end_idx+1]

            Xs_tr, ys_tr = build_sequences_matrix(X_tr, y_tr, k)

            # Val-Sequenzen erhalten k-1 Historie
            if len(X_tr) >= (k-1):
                X_val_aug = np.vstack([X_tr[-(k-1):], X_val])
                y_val_aug = np.hstack([y_tr[-(k-1):], y_val])
            else:
                X_val_aug, y_val_aug = X_val, y_val
            Xs_val, ys_val = build_sequences_matrix(X_val_aug, y_val_aug, k)

            # Skalierung (nur auf Training)
            scaler = StandardScaler().fit(Xs_tr.reshape(-1, Xs_tr.shape[-1]))
            Xs_tr  = scaler.transform(Xs_tr.reshape(-1, Xs_tr.shape[-1])).reshape(Xs_tr.shape).astype(np.float32)
            Xs_val = scaler.transform(Xs_val.reshape(-1, Xs_val.shape[-1])).reshape(Xs_val.shape).astype(np.float32)

            # Modell
            model = build_lstm_model(k, len(cols))
            es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

            # Warmup, um retracing zu vermeiden
            _ = model.predict(Xs_tr[:1], verbose=0)

            history = model.fit(
                Xs_tr, ys_tr,
                epochs=15, batch_size=16,
                validation_data=(Xs_val, ys_val),
                callbacks=[es],
                verbose=0
            )
            last_history = history.history

            p_val = model.predict(Xs_val, verbose=0).flatten().astype(np.float32)
            y_oof.extend(ys_val.tolist())
            p_oof.extend(p_val.tolist())

            # Aufräumen pro Fold
            tf.keras.backend.clear_session()

        if len(y_oof) == 0:
            continue

        y_oof = np.asarray(y_oof, dtype=np.int32)
        p_oof = np.asarray(p_oof, dtype=np.float32)

        # Platt (pure NumPy)
        platt_ab = platt_fit(p_oof, y_oof)
        p_oof_cal = platt_predict(p_oof, platt_ab)

        # F1-optimaler Threshold
        thresholds = np.linspace(0, 1, 101, dtype=np.float32)
        f1s = [f1_score(y_oof, (p_oof_cal >= t).astype(int), zero_division=0) for t in thresholds]
        best_idx = int(np.nanargmax(f1s))
        th = float(thresholds[best_idx])
        oof_f1 = float(f1s[best_idx])
        oof_auc = float(roc_auc_score(y_oof, p_oof) if len(np.unique(y_oof))>1 else 0.0)

        print(f"k={k}: OOF AUC={oof_auc:.3f} | OOF F1={oof_f1:.3f} | thr={th:.2f}")

        if oof_auc > best["auc"]:
            best.update({
                "k": k, "auc": oof_auc, "calibrator": platt_ab,
                "threshold": th, "oof_f1": oof_f1, "oof_auc": oof_auc, "history": last_history
            })

    # Final: Train mit bestem k
    k_best = best["k"]
    print(f"--> Gewähltes k: {k_best}")
    Xs_train_all, ys_train_all = build_sequences_matrix(Xtr, ytr, k_best)
    scaler_final = StandardScaler().fit(Xs_train_all.reshape(-1, Xs_train_all.shape[-1]))
    Xs_train_all = scaler_final.transform(Xs_train_all.reshape(-1, Xs_train_all.shape[-1])).reshape(Xs_train_all.shape).astype(np.float32)

    model_final = build_lstm_model(k_best, len(cols))
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    _ = model_final.predict(Xs_train_all[:1], verbose=0)
    history_final = model_final.fit(
        Xs_train_all, ys_train_all,
        epochs=15, batch_size=16,
        validation_split=0.1,
        callbacks=[es], verbose=0
    )

    results[fs_name] = {
        "model": model_final,
        "scaler": scaler_final,
        "calibrator": best["calibrator"],   # (a,b)
        "threshold": best["threshold"],
        "oof_f1": best["oof_f1"],
        "oof_auc": best["oof_auc"],
        "best_k": k_best,
        "history": best["history"]
    }



>>> Feature-Set: TECH
k=6: OOF AUC=0.591 | OOF F1=0.800 | thr=0.00
k=9: OOF AUC=0.633 | OOF F1=0.800 | thr=0.00
k=12: OOF AUC=0.681 | OOF F1=0.800 | thr=0.00
--> Gewähltes k: 12

>>> Feature-Set: MACRO
k=6: OOF AUC=0.616 | OOF F1=0.800 | thr=0.00
k=9: OOF AUC=0.559 | OOF F1=0.800 | thr=0.00
k=12: OOF AUC=0.609 | OOF F1=0.821 | thr=0.65
--> Gewähltes k: 6

>>> Feature-Set: INTEGRATED
k=6: OOF AUC=0.616 | OOF F1=0.800 | thr=0.00
k=9: OOF AUC=0.455 | OOF F1=0.800 | thr=0.00
k=12: OOF AUC=0.450 | OOF F1=0.800 | thr=0.00
--> Gewähltes k: 6


In [5]:
# Step4 (patched): Walk-Forward-Test nutzt den neuen calibrate_proba()-Wrapper
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix

for fs_name, res in results.items():
    print(f"\n>>> Walk-Forward-Test für {fs_name}")
    cols = feature_sets[fs_name]
    k = res["best_k"]

    # Kombiniere k-1 Historie
    Xtr_tail = Xtr_full[cols].values.astype(np.float32)[-(k-1):]
    Xte_vals = Xte_full[cols].values.astype(np.float32)
    X_combined = np.vstack([Xtr_tail, Xte_vals])

    # Sequenzen nur für Testbereich
    Xs_test = []
    for i in range(k-1, len(X_combined)):
        Xs_test.append(X_combined[i-k+1:i+1])
    Xs_test = np.asarray(Xs_test, dtype=np.float32)[:len(test_df)]

    # Skalierung
    Xs_test = res["scaler"].transform(Xs_test.reshape(-1, len(cols))).reshape(Xs_test.shape).astype(np.float32)

    # Roh-Probas
    p_test_raw = res["model"].predict(Xs_test, verbose=0).flatten().astype(np.float32)
    # Kalibrierte Probas (NumPy-Platt)
    p_test_cal = calibrate_proba(p_test_raw, res["calibrator"]).astype(np.float32)

    # Klassenvorhersage & Metriken
    y_true = yte_clf.values.astype(int)
    y_pred = (p_test_cal >= res["threshold"]).astype(int)
    test_f1 = f1_score(y_true, y_pred, zero_division=0)
    test_auc = roc_auc_score(y_true, p_test_cal) if len(np.unique(y_true))>1 else 0.0
    print(f"Test F1: {test_f1:.3f}, Test AUC: {test_auc:.3f}")

    res["y_true"] = y_true
    res["y_pred"] = y_pred
    res["p_test"] = p_test_cal
    res["test_f1"] = test_f1
    res["test_auc"] = test_auc



>>> Walk-Forward-Test für TECH
Test F1: 0.774, Test AUC: 0.502

>>> Walk-Forward-Test für MACRO
Test F1: 0.774, Test AUC: 0.565

>>> Walk-Forward-Test für INTEGRATED
Test F1: 0.774, Test AUC: 0.530


In [7]:
# Step5 (SAFE): Metriken, Signifikanz (DM & McNemar) ohne SciPy, JSON speichern
import json
import numpy as np
from math import erf, sqrt

def normal_cdf(z: float) -> float:
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

def chi2_sf_df1(x: float) -> float:
    # Survival-Funktion Chi^2(df=1) ≈ 2 * (1 - Φ(√x))
    if x < 0: 
        return 1.0
    return 2.0 * (1.0 - normal_cdf(sqrt(x)))

def dm_test_pvalue(e1: np.ndarray, e2: np.ndarray) -> float:
    # Einfache DM-Variante (ohne Newey-West auf Monatsdaten i.d.R. ok)
    d = e1 - e2
    sd = d.std(ddof=1)
    if sd == 0 or len(d) < 3:
        return 1.0
    t = d.mean() / (sd / sqrt(len(d)))
    # zweiseitig, Normal-Approx
    return float(2 * (1.0 - normal_cdf(abs(t))))

summary_rows = []
for fs_name, res in results.items():
    y_true = res["y_true"].astype(int)
    y_pred = res["y_pred"].astype(int)
    p      = res["p_test"].astype(float)

    # DM-Test (quadratischer Fehler) vs Always-Up
    y_always = np.ones_like(y_true)
    e_model  = (y_true - p)**2
    e_alw    = (y_true - y_always)**2
    dm_p_vs_always = dm_test_pvalue(e_model, e_alw)

    # McNemar vs Always-Up (Kontinuitätskorrektur)
    n01 = int(np.sum((y_true==1) & (y_pred==0)))  # Modell falsch, Always korrekt
    n10 = int(np.sum((y_true==0) & (y_pred==0)))  # Modell korrekt, Always falsch
    denom = n01 + n10
    if denom == 0:
        mcnemar_p = 1.0
    else:
        chi2_corr = (abs(n01 - n10) - 1)**2 / denom
        mcnemar_p = chi2_sf_df1(chi2_corr)

    metrics = {
        "threshold": float(res["threshold"]),
        "oof_f1": float(res["oof_f1"]),
        "oof_auc": float(res["oof_auc"]),
        "test_f1": float(res["test_f1"]),
        "test_auc": float(res["test_auc"]),
        "dm_p_vs_always": float(dm_p_vs_always),
        "dm_p_vs_rf": None,
        "mcnemar_p": float(mcnemar_p),
    }
    metrics_path = METRICS_DIR / f"lstm_clf_{fs_name}.json"
    metrics_path.write_text(json.dumps(metrics, indent=2), encoding="utf-8")
    print(f"OK: {metrics_path}")
    summary_rows.append({"featureset": fs_name, **metrics})


OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_TECH.json
OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_MACRO.json
OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_INTEGRATED.json


In [8]:
# Step5b: Metriken-JSON um permimp_path ergänzen (Schema-Ergänzung)
import json

for fs_name, res in results.items():
    metrics_path = METRICS_DIR / f"lstm_clf_{fs_name}.json"
    if metrics_path.exists():
        data = json.loads(metrics_path.read_text(encoding="utf-8"))
    else:
        data = {}
    data["permimp_path"] = res.get("permimp_path")
    # Platzhalter für cv_details_path (nicht erzeugt in diesem Notebook)
    data.setdefault("cv_details_path", None)
    metrics_path.write_text(json.dumps(data, indent=2), encoding="utf-8")
    print(f"aktualisiert: {metrics_path}")


aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_TECH.json
aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_MACRO.json
aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_INTEGRATED.json


In [9]:
# Step6 (revised): Explainability – Permutations-Importance robust (fix: build_sequences_full)
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

# deterministische Permutation
rng = np.random.RandomState(0)

def build_sequences_full(X: np.ndarray, y: np.ndarray, k: int):
    # Sequenzbau identisch zu build_sequences_matrix
    Xs, ys = [], []
    for i in range(k-1, len(X)):
        Xs.append(X[i-k+1:i+1])
        ys.append(y[i])
    return np.array(Xs), np.array(ys, dtype=int)

for fs_name, res in results.items():
    print(f"\nPermutation Importance für {fs_name}")
    cols = feature_sets[fs_name]
    k = res["best_k"]
    model = res["model"]
    scaler = res["scaler"]

    # Letztes Trainingsfenster (gesamtes Training) sequenzieren und skalieren
    Xs_train, ys_train = build_sequences_full(Xtr_full[cols].values, ytr_clf.values, k)
    Xs_train = scaler.transform(Xs_train.reshape(-1, len(cols))).reshape(Xs_train.shape)

    # Basis-AUC
    p_base = model.predict(Xs_train, verbose=0).flatten()
    auc_base = roc_auc_score(ys_train, p_base) if len(np.unique(ys_train))>1 else 0.0

    # Feature-weise Permutation (zeitlich synchron je Sequenz)
    importances = []
    n_seq = Xs_train.shape[0]
    for j, feat in enumerate(cols):
        X_perm = Xs_train.copy()
        idx = rng.permutation(n_seq)
        X_perm[:, :, j] = X_perm[idx, :, j]
        p_perm = model.predict(X_perm, verbose=0).flatten()
        auc_perm = roc_auc_score(ys_train, p_perm) if len(np.unique(ys_train))>1 else 0.0
        importances.append((feat, float(auc_base - auc_perm)))

    # Top-20 speichern
    importances.sort(key=lambda x: x[1], reverse=True)
    top20 = importances[:20]
    imp_df = pd.DataFrame(top20, columns=["feature", "importance"])
    permimp_path = METRICS_DIR / f"lstm_clf_{fs_name}_permimp.csv"
    imp_df.to_csv(permimp_path, index=False)
    results[fs_name]["permimp_path"] = str(permimp_path)
    print(f"OK: {permimp_path}")



Permutation Importance für TECH
OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_TECH_permimp.csv

Permutation Importance für MACRO
OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_MACRO_permimp.csv

Permutation Importance für INTEGRATED
OK: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_INTEGRATED_permimp.csv


In [10]:
# Step7: Plots erzeugen (Lernkurven, Confusion, kumulierte Rendite, Balken-Metriken, PermImp)
import matplotlib.pyplot as plt

for fs_name, res in results.items():
    print(f"\nPlots für {fs_name}")
    # Lernkurve (Loss train vs Val) vom letzten CV-Fold (falls gespeichert)
    if res["history"] is not None:
        loss = res["history"]["loss"]
        val_loss = res["history"]["val_loss"]
        plt.figure()
        plt.plot(loss, label="Training")
        plt.plot(val_loss, label="Validation")
        plt.title(f"Lernkurve LSTM ({fs_name})")
        plt.xlabel("Epoche")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(ROOT / "artifacts" / "reports" / f"23_learning_curve_{fs_name}.png")
        plt.close()
    # Confusion Matrix auf Testdaten
    cm = confusion_matrix(res["y_true"], res["y_pred"])
    plt.figure()
    plt.imshow(cm, cmap="Blues")
    plt.title(f"Konfusionsmatrix ({fs_name})")
    plt.xlabel("Vorhergesagt")
    plt.ylabel("Tatsächlich")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i,j], ha="center", va="center", color="black")
    plt.savefig(ROOT / "artifacts" / "reports" / f"23_confusion_{fs_name}.png")
    plt.close()
    # Kumulative Rendite: Strategie vs Always-Up
    returns = test_df["y_return_next_pct"].values / 100.0
    strat_factor = 1.0
    always_factor = 1.0
    strat_cum = []
    always_cum = []
    for r, pred in zip(returns, res["y_pred"]):
        always_factor *= (1 + r)
        if pred == 1:
            strat_factor *= (1 + r)
        strat_cum.append(strat_factor - 1)
        always_cum.append(always_factor - 1)
    plt.figure()
    plt.plot(strat_cum, label="Modell")
    plt.plot(always_cum, label="Always-Up")
    plt.title(f"Kumulierte Rendite ({fs_name})")
    plt.xlabel("Monat")
    plt.ylabel("Rendite (kumuliert)")
    plt.legend()
    plt.savefig(ROOT / "artifacts" / "reports" / f"23_cumret_{fs_name}.png")
    plt.close()
    # Balkendiagramm Metriken (Test-F1 vs Always-Up F1)
    # Hier: Vergleich der F1-Score des Modells vs Always-Up
    model_f1 = res["test_f1"]
    always_f1 = f1_score(res["y_true"], np.ones_like(res["y_true"]), zero_division=0)
    labels = ["Modell", "Always-Up"]
    plt.figure()
    plt.bar(labels, [model_f1, always_f1], color=["C0","C1"])
    plt.ylim(0,1)
    plt.title(f"Test-F1 ({fs_name})")
    plt.ylabel("F1-Score")
    plt.savefig(ROOT / "artifacts" / "reports" / f"23_bar_metrics_{fs_name}.png")
    plt.close()
    # Permutations-Importance Balkendiagramm (Top-20)
    imp_df = imp_df = pd.read_csv(ROOT / "artifacts" / "metrics" / f"lstm_clf_{fs_name}_permimp.csv")
    imp_df = imp_df.sort_values(by="importance", ascending=False).head(10)
    plt.figure(figsize=(6,4))
    plt.barh(imp_df["feature"][::-1], imp_df["importance"][::-1], color="C2")
    plt.title(f"Top-10 Perm. Importance ({fs_name})")
    plt.xlabel("Wertverlust in AUC")
    plt.savefig(ROOT / "artifacts" / "reports" / f"23_permimp_{fs_name}.png")
    plt.close()



Plots für TECH

Plots für MACRO

Plots für INTEGRATED


In [11]:
# Step8 (revised): Modelle im Keras-Native-Format speichern (.keras) + Pfad in Metriken aktualisieren
import json
from pathlib import Path

for fs_name, res in results.items():
    # .keras-Format (empfohlen)
    model_path_keras = (MODELS_DIR := (ROOT / "artifacts" / "models"))
    model_path_keras.mkdir(parents=True, exist_ok=True)
    model_path_keras = model_path_keras / f"lstm_clf_{fs_name}.keras"
    res["model"].save(model_path_keras)
    print(f"Gespeichert (.keras): {model_path_keras}")

    # Forecasts (CSV)
    df_forecast = pd.DataFrame({
        "date": test_df.index,
        "y_true": res["y_true"],
        "y_pred_prob": res["p_test"],
        "y_pred": res["y_pred"]
    }).set_index("date")
    forecast_path = ROOT / "artifacts" / "forecasts" / f"lstm_clf_{fs_name}.csv"
    forecast_path.parent.mkdir(parents=True, exist_ok=True)
    df_forecast.to_csv(forecast_path)
    print(f"Forecasts: {forecast_path}")

    # Metriken-JSON um model_path erweitern/anpassen
    metrics_path = ROOT / "artifacts" / "metrics" / f"lstm_clf_{fs_name}.json"
    if metrics_path.exists():
        data = json.loads(metrics_path.read_text(encoding="utf-8"))
    else:
        data = {}
    data["model_path"] = str(model_path_keras)
    metrics_path.write_text(json.dumps(data, indent=2), encoding="utf-8")
    print(f"Metriken aktualisiert: {metrics_path}")


Gespeichert (.keras): C:\Users\gamer\Desktop\AktienPrognose\artifacts\models\lstm_clf_TECH.keras
Forecasts: C:\Users\gamer\Desktop\AktienPrognose\artifacts\forecasts\lstm_clf_TECH.csv
Metriken aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_TECH.json
Gespeichert (.keras): C:\Users\gamer\Desktop\AktienPrognose\artifacts\models\lstm_clf_MACRO.keras
Forecasts: C:\Users\gamer\Desktop\AktienPrognose\artifacts\forecasts\lstm_clf_MACRO.csv
Metriken aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_MACRO.json
Gespeichert (.keras): C:\Users\gamer\Desktop\AktienPrognose\artifacts\models\lstm_clf_INTEGRATED.keras
Forecasts: C:\Users\gamer\Desktop\AktienPrognose\artifacts\forecasts\lstm_clf_INTEGRATED.csv
Metriken aktualisiert: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_INTEGRATED.json


In [12]:
# Step9: Zusammenfassung aller Feature-Sets (CSV und Übersicht-Plot)
summary_df = pd.DataFrame(summary_rows)
# CSV speichern
summary_csv = ROOT / "artifacts" / "metrics" / "lstm_clf_summary.csv"
summary_csv.parent.mkdir(parents=True, exist_ok=True)
summary_df.to_csv(summary_csv, index=False)
print(f"Gespeichert Übersicht CSV: {summary_csv}")
# Balkendiagramm: F1 und AUC pro Feature-Set
x = np.arange(len(summary_df))
width = 0.35
fig, ax = plt.subplots()
ax.bar(x - width/2, summary_df["test_f1"], width, label="F1-Score")
ax.bar(x + width/2, summary_df["test_auc"], width, label="AUC")
ax.set_xticks(x)
ax.set_xticklabels(summary_df["featureset"])
ax.set_ylim(0,1)
ax.set_ylabel("Wert")
ax.set_title("Test F1 vs AUC nach Feature-Set")
ax.legend()
plt.savefig(ROOT / "artifacts" / "reports" / "23_summary.png")
plt.close()


Gespeichert Übersicht CSV: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\lstm_clf_summary.csv
